In [2]:
from pandas import read_csv
from pathlib import Path
import pandas as pd
import numpy as np
dir_data_2023 = Path('data/processed_2023')
dir_data_2024 = Path('data/processed_2024')
dir_data_merged = Path('data/processed_merged')
def parse_filename(file_path):
    parts = file_path.stem.split('_')
    sample_indx = int(parts[0])
    label = parts[1]
    start = parts[2]
    end = parts[4]
    return sample_indx, label, start, end
def sum_dataset(dir_data):
    dir_data = Path(dir_data)  # Ensure dir_data is a Path object
    filespath = dir_data.glob('*.csv')
    all_dataset = {}

    for path in filespath:
        _, label,_,_= parse_filename(path)
        if label in all_dataset:
            all_dataset[label] += 1
        else:
            all_dataset[label] = 1

    # Creating a DataFrame from the dictionary
    sum_df = pd.DataFrame(list(all_dataset.items()), columns=['Label', 'Count'])
    return sum_df

In [6]:
dir_data_2024 = sum_dataset(dir_data_2024)
# sort the dataframe by count
dir_data_2024 = dir_data_2024.sort_values(by='Count', ascending=False)
print(dir_data_2024)

        Label  Count
1        CTC4  15702
2    RB966928   5875
6     CTC9003   4791
8     CTC9001   4372
0     CTC9002   3130
5    RB975242   2007
4      CV7870   1797
12   RB867515   1745
7    RB975201   1404
10   RB985476   1310
13    CTC2994   1297
9       CTC20   1229
3    RB975033   1169
11  CTC9005HP   1150


In [ ]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

def merge_csv_files_based_on_dates(dir1, dir2, output_dir):
    dir1 = Path(dir1)
    dir2 = Path(dir2)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Create a dictionary to track files by identifiers
    file_map = {}

    # Process all files in both directories
    for directory in [dir1, dir2]:
        for file_path in directory.glob('*.csv'):
            sample_indx, label, start, end= parse_filename(file_path)
            identifier = f'{sample_indx}_{label}'
            if identifier in file_map:
                file_map[identifier].append(file_path)
            else:
                file_map[identifier] = [file_path]

    # Merge files that have the same identifier
    bar = tqdm(file_map.items(), desc='Merging files')
    for identifier, files in bar:
        if len(files) > 1:

            dataframes = [pd.read_csv(f) for f in files]
            merged_df = pd.concat(dataframes, ignore_index=True)

            merged_df['SourceFile_Start_Date'] = merged_df['SourceFile'].apply(extract_start_date)
            merged_df.sort_values(by='SourceFile_Start_Date', inplace=True)
            merged_df.drop(columns=['SourceFile_Start_Date'], inplace=True) 

            merged_filename = f"{identifier}.csv"
            merged_df.to_csv(output_dir / merged_filename, index=False)
            # print(f"Merged file saved: {output_dir / merged_filename}")
        bar.set_postfix_str(f'Num of processed files: {len(file_map)}')
def extract_start_date(source_file):
    start_date = source_file.split('_to_')[0]
    return pd.to_datetime(start_date)
# Example usage
merge_csv_files_based_on_dates(
    'data/processed_2023',
    'data/processed_2024',
    'data/processed_merged'
)

In [13]:
weekly_data = list(df.groupby('SourceFile'))

In [25]:
i = 6
time = weekly_data[i][0]
data = weekly_data[i][1].iloc[:,4:-2]
print(f"Time: {time}")
data

Time: 2023-05-13_to_2023-05-19


,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,Band_12
1104,6,9,18,24,34,57,61,62,61,65,51,39
1105,6,9,17,21,31,58,64,64,65,65,49,35
1106,6,8,16,17,29,58,66,67,68,65,48,33
1107,6,8,15,18,28,57,66,68,67,66,48,34
1108,6,8,16,18,29,57,66,66,67,66,49,35
...,...,...,...,...,...,...,...,...,...,...,...,...
1283,6,8,17,21,31,55,63,63,64,64,51,39
1284,6,8,16,18,29,56,64,65,66,65,49,36
1285,7,10,18,20,30,58,65,66,68,69,52,37
1286,4,8,16,18,28,58,68,69,72,72,48,27


# To dataset

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
def random_time_bins_selction(band_data):
    dates = pd.to_datetime(band_data.time.values)

    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame({'date': dates})

    # Extract year and month for grouping
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month

    # Group by year and month, then randomly select one observation from each group
    return df.groupby(['year', 'month']).apply(lambda x: np.random.choice(x.index)).tolist()
def get_data_time(self, df):
    

    all_month_data = np.zeros((12, self.num_bands, self.patch_height, self.patch_width))
    all_day = np.array(pd.date_range(start=f'{netcdf.patch_year}-01-01', end=f'{int(netcdf.patch_year) + 1}-01-01', freq=self.group_freq).dayofyear)[:-1]

    data = []
    day = []
    month    = []
    for band_id, band in enumerate(self.bands):
        # Load band data
        band_data    = xr.open_dataset(xr.backends.NetCDF4DataStore(netcdf[band]))

        selected_idx = random_time_bins_selction(band_data)

        d = band_data.to_array().values[:,selected_idx]
        t = band_data.time.values[selected_idx]
        
        expand_ratio = int(BANDS[band] / BANDS[REFERENCE_BAND])
        if expand_ratio != 1:
            d = np.repeat(d, expand_ratio, axis=-1)
            d = np.repeat(d, expand_ratio, axis=-2)

        data.append(d)
        day.append(pd.to_datetime(t).dayofyear)
        month.append(pd.to_datetime(t).month)

    data = np.concatenate(data).transpose(1, 0, 2, 3)
    month = np.stack(month).mean(axis = 0,dtype = int)

    day = np.stack(day).mean(axis=0)

    all_month_data[month-1] = data
    all_day[month-1] = day
    return all_month_data, all_day/365

In [3]:
path_data = list(dir_data_merged.glob('*.csv'))[0]
df = pd.read_csv(path_data)

def extract_interesting_data(df, bands):
    # Extract the starting date from the SourceFile column
    df['StartDate'] = df['SourceFile'].apply(lambda x: x.split('_to_')[0])
    
    # Convert StartDate to datetime to extract the week of the year
    df['WeekOfYear'] = pd.to_datetime(df['StartDate']).dt.isocalendar().week
    
    # Calculate week of the year divided by 52 to get the fraction of the year
    df['WeekFraction'] = df['WeekOfYear'] / 52
    
    # Filter the dataframe to include only the specified bands and the new WeekFraction column
    columns_of_interest = bands + ['WeekFraction']
    filtered_df = df[columns_of_interest]
    
    return filtered_df



In [2]:
import random
from tqdm import tqdm
import numpy as np
from pathlib import Path
from scipy.interpolate import interp1d
def interpolate_along_axis(data, axis=1):
    # Interpolate along the specified axis for each slice in the other dimensions
    shape = data.shape
    # Iterate over the first and third dimension, interpolating along the second
    for i in range(shape[0]):
        for j in range(shape[2]):
            # Extract the line to interpolate
            line = data[i, :, j]
            # Find indices where data is non-zero
            if np.any(line):  # Check if there's at least some non-zero data to interpolate
                indices = np.arange(len(line))
                good_data = line != 0
                # Create an interpolation function based on non-zero data
                f = interp1d(indices[good_data], line[good_data], kind='linear', bounds_error=False, fill_value="extrapolate")
                # Interpolate missing data
                data[i, :, j] = f(indices)
            # Optionally handle cases where the entire line is zero by setting a default or carrying forward last known good data

    return data
def extract_interesting_data(df, bands):
    # Extract the starting date from the SourceFile column
    df['StartDate'] = df['SourceFile'].apply(lambda x: x.split('_to_')[0])
    
    # Convert StartDate to datetime to extract the week of the year
    df['WeekOfYear'] = pd.to_datetime(df['StartDate']).dt.isocalendar().week
    
    # Calculate week of the year divided by 52 to get the fraction of the year
    df['WeekFraction'] = df['WeekOfYear'] / 52
    
    df['MonthOfYear'] = pd.to_datetime(df['StartDate']).dt.month
    df['MonthOfYear'] = df['MonthOfYear'] / 12
    # Filter the dataframe to include only the specified bands and the new WeekFraction column
    columns_of_interest = bands + ['WeekFraction'] + ['MonthOfYear'] + ['StartDate']
    filtered_df = df[columns_of_interest]
    
    return filtered_df

def split_data(all_dataset, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    if train_ratio + val_ratio + test_ratio != 1:
        raise ValueError("Sum of ratios must be equal to 1")

    # Initialize dictionaries for train, val, and test sets
    train_set = {}
    val_set = {}
    test_set = {}

    for label, paths in all_dataset.items():
        # Shuffle paths to ensure randomness
        random.shuffle(paths)
        
        # Calculate split indices
        n_total = len(paths)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)
        
        # Create splits
        train_paths = paths[:n_train]
        val_paths = paths[n_train:n_train + n_val]
        test_paths = paths[n_train + n_val:]

        # Store paths by label in the respective dictionaries
        train_set[label] = train_paths
        val_set[label] = val_paths
        test_set[label] = test_paths

    return train_set, val_set, test_set
def parse_filename(file_path):
    parts = file_path.stem.split('_')
    sample_indx = int(parts[0])
    label = parts[1]
    return sample_indx, label

def sum_dataset(dir_data):
    dir_data = Path(dir_data)  # Ensure dir_data is a Path object
    filespath = dir_data.glob('*.csv')
    all_dataset = {}

    for path in filespath:
        _, label= parse_filename(path)
        if label in all_dataset:
            all_dataset[label].append(path)
        else:
            all_dataset[label] = []
    return all_dataset
def remove_zero_rows(df, band_columns ):
    # Specify the band columns to check for zeros
    
    
    # Check if all values in these columns are zero
    mask = (df[band_columns] == 0).all(axis=1)
    
    # Filter out rows where all band values are zero
    filtered_df = df[~mask]
    
    return filtered_df
def compute_monthly_median(df):
    # Create a copy of the dataframe to avoid modifying the original inadvertently
    df_copy = df.copy()

    # Convert the 'StartDate' column to datetime
    df_copy['StartDate'] = pd.to_datetime(df_copy['StartDate'])

    # Set 'StartDate' as the index of the dataframe
    df_copy.set_index('StartDate', inplace=True)
    
    # Resample data by month and calculate the median
    monthly_median = df_copy.resample('M').median()

    return monthly_median
def compute_median_of_each_month(filtered_df, interested_bands):
    allmonth_data = {}
    all_month = []
    for start_date, group in filtered_df.groupby('StartDate'):

        year = pd.to_datetime(start_date).year
        month = pd.to_datetime(start_date).month
        # print(start_date)
        # print(year)
        # print(month)
        # print(group[interested_bands].values.shape)
        key = f'{year}_{month}_{month/12}'
        data = group[interested_bands].values
        if key in allmonth_data:
            allmonth_data[key].append(data)
        else:
            allmonth_data[key] = [data]
    for key, data in allmonth_data.items():
        data = np.stack(data)
        median = np.median(data, axis=0)
        #  replace the value in the key with the median value
        allmonth_data[key] = median
        all_month.append(int(key.split('_')[1])/12)
    return allmonth_data, np.array(all_month)
save_dir = Path('data/ctc_dataset')

source_dir = Path('data/processed_merged')
all_dataset = sum_dataset(source_dir)
train_set, val_set, test_set = split_data(all_dataset)

splited_dataset = {'train': train_set, 'val': val_set, 'test': test_set}
for mode in ['train', 'val', 'test']:
    data_paths_dict = splited_dataset[mode]
    all_labels = list(data_paths_dict.keys())
    bar = tqdm(all_labels, desc=f'Processing {mode} data')
    for label in bar:
        data_paths = data_paths_dict[label]
        for i, path in enumerate(data_paths):

            save_path_data = save_dir / mode / label / f'data_{i}'
            save_path_time = save_dir / mode / label / f'time_{i}'
            save_path_data.parent.mkdir(parents=True, exist_ok=True)
            save_path_time.parent.mkdir(parents=True, exist_ok=True)

            if save_path_data.with_suffix('.npy').exists() and save_path_time.with_suffix('.npy').exists():
                continue
            source_df = pd.read_csv(path)
            if source_df.empty:
                continue
            try:
                interested_bands = [f'Band_{i}' for i in range(1, 13)]

                filtered_df = extract_interesting_data(source_df, interested_bands)

                filtered_df, time = compute_median_of_each_month(filtered_df, interested_bands)

                data = np.stack(list(filtered_df.values()),axis = 1)
                assert data.shape[1:] == (16,12), f"Data for {label} is not complete, as it has shape of {data.shape}"

                # #     # Save the data to .npy files
                np.save(save_path_data.with_suffix('.npy'), data)
                np.save(save_path_time.with_suffix('.npy'), time)
                bar.set_postfix_str(f'Processed {i} data')
            except Exception as e:
                print(f"Error processing {path}")
                print(e)
                continue
    #         break
    #     break
    # break

    


In [3]:
def count_data_samples(splited_dataset):
    """
    Count the number of data samples in each of the train, test, and val sets across all labels.
    
    Args:
    splited_dataset (dict): A dictionary where the keys are 'train', 'test', 'val',
                            and values are dictionaries with labels as keys and lists of data paths as values.
                            
    Returns:
    dict: A dictionary with keys 'train', 'test', 'val' and values being the total number of data samples.
    """
    # Initialize counts
    counts = {'train': 0, 'test': 0, 'val': 0}
    
    # Iterate over each set in the dictionary ('train', 'test', 'val')
    for set_name in ['train', 'test', 'val']:
        # Check if the set name key exists in the dictionary
        if set_name in splited_dataset:
            # Iterate over each label in the set
            for label in splited_dataset[set_name]:
                # Accumulate the count of data paths for each label
                counts[set_name] += len(splited_dataset[set_name][label])
    
    return counts
counts = count_data_samples(splited_dataset)

In [4]:

counts

{'train': 37564, 'test': 4710, 'val': 4689}

In [5]:
37564 + 4710 +4689

46963

In [5]:
import pandas as pd
import numpy as np

def pick_spread_months(df, num_rows=12):
    # Ensure dataframe is sorted by 'MonthOfYear'
    df_sorted = df.sort_values(by='MonthOfYear')

    # Calculate bin size
    num_bins = num_rows  # aiming to pick one from each bin if possible
    bin_size = len(df_sorted) // num_bins

    # Select one row from each bin
    selected_indices = []
    for i in range(num_bins):
        start_idx = i * bin_size
        if i == num_bins - 1:  # For the last bin, include all remaining rows
            end_idx = len(df_sorted)
        else:
            end_idx = start_idx + bin_size

        # Pick a row randomly from each bin or pick the middle one to ensure spread
        middle_idx = (start_idx + end_idx) // 2
        selected_indices.append(middle_idx)

    # Select the rows
    selected_rows = df_sorted.iloc[selected_indices]

    # Since bins might not perfectly align with the number of desired rows,
    # pick the closest unique months till you get the desired count
    final_selected = selected_rows.drop_duplicates(subset='MonthOfYear').head(num_rows)

    # If we have fewer than needed because of duplicates, fill in from other rows not picked initially
    if len(final_selected) < num_rows:
        extra_needed = num_rows - len(final_selected)
        additional_rows = df_sorted[~df_sorted.index.isin(final_selected.index)].drop_duplicates(subset='MonthOfYear').head(extra_needed)
        final_selected = pd.concat([final_selected, additional_rows])

    return final_selected

# Example usage:
# df = pd.read_csv('path_to_your_file.csv')  # Load your dataframe here
selected_rows = pick_spread_months(filtered_df)
# print(selected_rows)

In [7]:
selected_rows['MonthOfYear'] = selected_rows['MonthOfYear'] * 12
selected_rows

,WeekFraction,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_9,Band_10,Band_11,Band_12,MonthOfYear
1,0.038462,12.878788,16.030303,25.393939,20.575758,37.939394,99.424242,131.151515,125.818182,138.848485,133.727273,76.757576,43.272727,1.0
4,0.115385,6.212121,8.575758,17.303030,13.757576,27.454545,79.060606,97.000000,96.787879,102.393939,101.212121,55.666667,30.727273,2.0
7,0.192308,6.212121,7.575758,15.818182,12.272727,25.090909,75.333333,93.060606,91.090909,98.909091,97.121212,50.060606,26.393939,3.0
13,0.326923,5.772727,8.772727,16.469697,13.121212,26.363636,74.439394,93.636364,94.242424,99.787879,98.075758,50.439394,25.939394,4.0
16,0.384615,5.272727,9.515152,18.772727,15.606061,30.863636,79.287879,96.530303,100.196970,103.106061,101.969697,53.333333,28.151515,5.0
18,0.423077,5.666667,9.272727,17.984848,14.818182,28.833333,73.984848,89.909091,92.424242,95.984848,94.984848,51.121212,27.242424,6.0
22,0.500000,5.227273,9.803030,18.984848,16.318182,31.757576,73.984848,88.984848,91.636364,95.590909,94.909091,47.939394,25.227273,6.5
25,0.557692,6.909091,11.333333,20.378788,20.227273,34.666667,70.166667,83.818182,86.090909,90.424242,89.954545,55.439394,31.196970,7.0
28,0.615385,6.969697,11.212121,19.242424,16.878788,30.878788,72.333333,89.515152,91.424242,95.666667,93.787879,51.242424,28.030303,8.0
33,0.711538,11.484848,13.484848,21.969697,20.969697,34.909091,70.545455,82.545455,83.090909,89.757576,88.909091,58.212121,32.727273,9.0


In [66]:
# Group the data by the week fraction, and print the first group
for i, (week_fraction, group) in enumerate(filtered_df.groupby('WeekFraction')):
    if i == 0:
        print(f'Week Fraction: {week_fraction}')
        # print(group)
        # take the median of the group
        print(group.mean())
        print(group.median())
        break


Week Fraction: 0.019230769230769232
Band_1            9.222222
Band_2           10.444444
Band_3           19.666667
Band_4           17.527778
Band_5           30.833333
Band_6           79.861111
Band_7           99.888889
Band_8           98.333333
Band_9          105.388889
Band_10         103.250000
Band_11          56.638889
Band_12          33.777778
WeekFraction      0.019231
dtype: float64
Band_1            8.000000
Band_2            9.000000
Band_3           18.000000
Band_4           13.500000
Band_5           27.000000
Band_6           80.000000
Band_7          102.000000
Band_8          100.000000
Band_9          108.000000
Band_10         106.000000
Band_11          50.000000
Band_12          27.500000
WeekFraction      0.019231
dtype: float64
